# ADT Tutorial

Note: If this is your first ADT tutorial, we suggest going through the ADT quickstart tutorial first. This tutorial will not explain the basics that are covered in the quickstart.

In this tutorial, we will explain how to generate 3D pointclouds from the ADT depth maps. This includes unprojecting points using the camera model to get the 3D ray associated with each pixel, using the depth map to compute each point's 3D coordinates in the camera frame, using the Aria pose and calibration to compute the point coordinates in the Scene frame, and finally using the RGB images to colorize the pointcloud.

### Running in Google Colab

To run this in google colab, go to [ADT Depth Map to Pointcloud Tutorial](https://colab.research.google.com/github/facebookresearch/projectaria_tools/blob/main/projects/AriaDigitalTwinDatasetTools/examples/adt_depth_maps_to_pointcloud_tutorial.ipynb)

### Setup notebook

In [ ]:
# Specifics for Google Colab
google_colab_env = 'google.colab' in str(get_ipython())
if google_colab_env:
    print("Running from Google Colab, installing projectaria_tools and getting sample data")
    !pip install projectaria-tools
    # These versions of numpy and pandas-gbq are compatible with the rest of the colab environment
    !pip install numpy>=1.23 pandas-gbq==0.19


In [ ]:
import numpy as np
import os
import sys
import subprocess
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import plotly.graph_objects as go
from math import tan
import random

from projectaria_tools.core.stream_id import StreamId
from projectaria_tools.core import calibration
from projectaria_tools.projects.adt import (
   AriaDigitalTwinDataProvider,
   AriaDigitalTwinSkeletonProvider,
   AriaDigitalTwinDataPathsProvider,
   bbox3d_to_line_coordinates,
   bbox2d_to_image_coordinates,
   utils as adt_utils,
)

### Download the example sequence

The following code cell will directly download the [ADT sample dataset](https://www.projectaria.com/async/sample/download/?bucket=adt&filename=aria_digital_twin_test_data.zip).  

In [ ]:
if google_colab_env:
    adt_sample_path = "./adt_sample_data"
else:
    adt_sample_path = "/tmp/adt_sample_data"

data_sequence_url = "https://www.projectaria.com/async/sample/download/?bucket=adt&filename=aria_digital_twin_test_data_v2.zip"
command_list = [
    f"mkdir -p {adt_sample_path}",
    # Download sample data
    f'curl -o {adt_sample_path}/adt_sample_data.zip -C - -O -L "{data_sequence_url}"',
    # Unzip the sample data
    f"unzip -o {adt_sample_path}/adt_sample_data.zip -d {adt_sample_path}"
]
sequence_path = f"{adt_sample_path}/Apartment_release_golden_skeleton_seq100_10s_sample_M1292"

# Execute the commands for downloading dataset
if google_colab_env:
    for command in command_list:
        !$command
else:
    for command in command_list:
        subprocess.run(command, shell=True, check=True)

### Load data

In [ ]:
paths_provider = AriaDigitalTwinDataPathsProvider(sequence_path)
data_paths = paths_provider.get_datapaths()
gt_provider = AriaDigitalTwinDataProvider(data_paths)

## Get calib



In [ ]:
stream_id = StreamId("214-1")
camera_calibration = gt_provider.get_aria_camera_calibration(stream_id)
T_Device_Cam = camera_calibration.get_transform_device_camera()

## Set timestamps

In [ ]:
img_timestamps_ns_all = gt_provider.get_aria_device_capture_timestamps_ns(stream_id)
ts1 = img_timestamps_ns_all[10]
ts2 = img_timestamps_ns_all[len(img_timestamps_ns_all)-10]
img_timestamps_ns = [ts1, ts2]
print("selected timestamps: ")
for ts in img_timestamps_ns:
  print(ts * 1e-9, "s")


## Load depth images

In [ ]:
SKIP_N_PIXELS = 10

# store a list of points in the scene frame for each image
points_in_scene = []

# also store a list of rgb colors for each image, where each col corresponds to a point in the pointcloud
rgb_cols = []

for timestamp in img_timestamps_ns:
  if timestamp < gt_provider.get_start_time_ns() or timestamp > gt_provider.get_end_time_ns():
    print(f"WARNING: timestamp outside of GT domain")
    continue

  aria_pose_with_dt = gt_provider.get_aria_3d_pose_by_timestamp_ns(timestamp)
  if not aria_pose_with_dt.is_valid:
    print(f"WARNING: No Aria poses for timestamp {timestamp}, skipping image")
    continue

  T_Scene_Device = aria_pose_with_dt.data().transform_scene_device
  T_Scene_Cam = T_Scene_Device @ T_Device_Cam

  depth = gt_provider.get_depth_image_by_timestamp_ns(timestamp, stream_id).data().to_numpy_array()
  rgb = gt_provider.get_aria_image_by_timestamp_ns(timestamp, stream_id).data().to_numpy_array()
  u_max = rgb.shape[1]
  v_max = rgb.shape[0]

  # iterate through all pixels
  counter = 0.0
  img_points_in_scene = []
  img_cols = []
  for u in range(u_max):
    for v in range(v_max):
      counter = counter + 1.0

      # skip every N image pixel to speed things up
      if counter % SKIP_N_PIXELS != 0.0:
        continue
      ray = camera_calibration.unproject([u,v])
      if ray is not None:
        d = depth[v,u] / 1000 # diving by 1000 to convert from mm to m
        p_in_cam = d * ray
        p_in_scene = T_Scene_Cam @ p_in_cam
        img_points_in_scene.append(p_in_scene)

        # Note that our API calls go by foo(u,v) or foo(x,y)
        # However, when converting to a numpy array, the array must be indexed by (row, col),
        # hence the opposite call here vs unproject
        img_cols.append(rgb[v][u])

  # convert from lists to ndarrays
  points_in_scene.append(np.stack(img_points_in_scene))
  rgb_cols.append(np.stack(img_cols))

## View Results

### Setup ReRun

In [ ]:
import rerun as rr
rr.init(
    "ADT Pointcloud Viewer",
    recording_id=None,
    spawn=True,
    default_enabled=True,
    strict=False,
)
rr.log("world", rr.ViewCoordinates.RIGHT_HAND_Y_UP, timeless=True)
rec = rr.memory_recording()

### View different depth images using different colors to show alignment between the two

In [ ]:
import random
for i in range(len(points_in_scene)):
  rand_col = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
  rr.log(
        "world/points/" + str(img_timestamps_ns[i]),
        rr.Points3D(points_in_scene[i], colors=[rand_col]),
        timeless=True,
    )
rec

### View depth images with color from the RGB

In [ ]:
# we can use the colors we extracted from the RGB images above to get a colored pointcloud
for i in range(len(points_in_scene)):
  rr.log(
        "world/points/" + str(img_timestamps_ns[i]),
        rr.Points3D(points_in_scene[i], colors=rgb_cols[i]),
        timeless=True,
    )
rec